In [2]:
import pandas as pd
import numpy as np
import nltk
from nltk import sent_tokenize,word_tokenize
from collections import defaultdict
import re

In [6]:
text='i am happy because i am learning .'
corpus = word_tokenize(text)

In [862]:
def create_n_grams(text,N, start_tags=True,end_tags=True):

    t=sent_tokenize(text)
    ngrams=[]
    for sent in t:
        tok_sent=word_tokenize(sent)
        if start_tags and end_tags:
            tok_sent=['<s>']*(N-1)+tok_sent+['</s>']
        elif start_tags==True and end_tags==False:
            tok_sent=['<s>']*(N-1)+tok_sent
        elif start_tags==False and end_tags==True:
            tok_sent=tok_sent+['</s>']
        for i in range(len(tok_sent)-N+1):
            ngrams.append(tuple(tok_sent[i:i+N]))
    return ngrams

In [883]:
bi_grams=create_n_grams(text,2, start_tags=True,end_tags=True)

In [884]:
bi_grams

[('<s>', 'i'),
 ('i', 'am'),
 ('am', 'happy'),
 ('happy', 'because'),
 ('because', 'i'),
 ('i', 'am'),
 ('am', 'learning'),
 ('learning', '.'),
 ('.', '</s>')]

In [885]:
tri_grams=create_n_grams(text,3, start_tags=True,end_tags=True)

In [886]:
tri_grams

[('<s>', '<s>', 'i'),
 ('<s>', 'i', 'am'),
 ('i', 'am', 'happy'),
 ('am', 'happy', 'because'),
 ('happy', 'because', 'i'),
 ('because', 'i', 'am'),
 ('i', 'am', 'learning'),
 ('am', 'learning', '.'),
 ('learning', '.', '</s>')]

In [887]:
te=[]
for i in range(len(bi_grams)):
    te.append(" ".join(bi_grams[i]))
vocab=sorted(set(" ".join(te).split()))
#to get the unique words in the corpus

In [888]:
#to calculate the probability matrix
d=defaultdict(list)
for i in range(len(bi_grams)):
    try:
        d[bi_grams[i]].append(tri_grams[i+1][-1])
    except IndexError:
        d[bi_grams[i]].append(tri_grams[i][-1])

In [889]:
#initialize the matrix
matrix=np.zeros(shape=(len(d.keys()),len(vocab)))

In [890]:
#fill in the matrix with the count of every word appearing after 
#each ngram
keys=list(d.keys())
for key in keys:
    gram=keys.index(key)
    for word in d[key]:
        count=0
        idx=vocab.index(word)
        count+=1
        matrix[gram,idx]=count

In [891]:
sum_rows=matrix.sum(axis=1,keepdims=True)
matrix=np.divide(matrix,sum_rows)

In [892]:
df=pd.DataFrame(matrix,index=list(d.keys()),columns=vocab)

In [896]:
df

,.,</s>,<s>,am,because,happy,i,learning
"(<s>, i)",0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
"(i, am)",0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.5
"(am, happy)",0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
"(happy, because)",0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
"(because, i)",0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
"(am, learning)",1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(learning, .)",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
"(., </s>)",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
